Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [155]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [156]:
PROBLEM_SIZE = 10
NUM_SETS = 30
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])
SETS

(array([ True, False, False,  True, False,  True, False, False, False,
        False]),
 array([False, False, False,  True, False, False, False, False,  True,
        False]),
 array([False,  True, False, False,  True,  True,  True, False, False,
         True]),
 array([False,  True,  True, False,  True, False, False, False, False,
        False]),
 array([ True, False,  True, False, False, False, False, False,  True,
        False]),
 array([False,  True, False,  True, False,  True,  True, False,  True,
         True]),
 array([False, False, False, False, False,  True,  True, False, False,
        False]),
 array([False, False, False, False, False, False, False,  True, False,
        False]),
 array([ True, False,  True, False, False, False,  True, False, False,
        False]),
 array([False, False, False, False,  True, False, False,  True,  True,
        False]),
 array([ True, False, False,  True, False, False, False, False,  True,
        False]),
 array([False, False, False, Fal

In [157]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def distance_A_star(state, w=0.5):
     
    dist = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    
    # optimization: try to minimize overlap
    n_overlap = np.sum(np.sum([SETS[i] for i in state.taken], axis=0) > 1)
    
    return w*dist + (1-w)*n_overlap

In [158]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [159]:
def search(distance_func=None):

    # frontier = PriorityQueue()
    frontier = SimpleQueue()
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((distance_func(state), state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        # print(current_state)
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((distance_func(new_state), new_state))
            # print(f'taken: {new_state.taken}')
            # print(f'Not taken: {new_state.not_taken}')
            # print()
        _, current_state = frontier.get()

    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
    )
    return counter, current_state

In [160]:
# set distance function:    distance   distance_A_star
counter, current_state = search(distance_func=distance)

print('SOLUTION:')
print()
for s in current_state.taken:
    print(f'{s}: {SETS[s]}')

Solved in 943 steps (3 tiles)
SOLUTION:

16: [False  True  True  True  True  True False  True  True False]
0: [ True False False  True False  True False False False False]
2: [False  True False False  True  True  True False False  True]


In [161]:
# set distance function:    distance   distance_A_star
counter, current_state = search(distance_func=distance_A_star)

print('SOLUTION:')
print()
for s in current_state.taken:
    print(f'{s}: {SETS[s]}')

Solved in 943 steps (3 tiles)
SOLUTION:

16: [False  True  True  True  True  True False  True  True False]
0: [ True False False  True False  True False False False False]
2: [False  True False False  True  True  True False False  True]
